In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.optimize import root
import scipy.linalg as la
import sys
if ".." not in sys.path: sys.path.append("..")
from common import constants, cosmology

In [7]:
def alpha_YM(q, q0, alpha0, N, N_f=6):
    # from Schwartz QFT and the SM
    C_F = (N**2 - 1) / (2*N)
    T_F = 1/2
    C_A = N
    L = np.log(q / q0)
    beta0 = 11/3 * C_A - 4/3 * T_F * N_f
    beta1 = 34/3 * C_A**2 - 20/3 * C_A * T_F * N_f - 4 * C_F * T_F * N_f
    return (
        alpha0 
        - alpha0**2 / (2*np.pi) * beta0 * L
        + alpha0**3 / (8*np.pi**2) * (-beta1 * L + 2 * beta0**2 * L**2)
    )

def alpha2_running(q):
    return alpha_YM(q, constants.M_Z, constants.alpha_2, 2)

def alpha3_running(q):
    return alpha_YM(q, constants.M_Z, constants.alpha_3, 3)

In [8]:
def alpha1_running(q):
    e0_sq = 4*np.pi*constants.alpha_1
    q0 = constants.M_Z
    # 1 loop :(
    e_sq = 1 / (1 / e0_sq - 1 / (12*np.pi**2) * np.log(q**2 / q0**2))
    return e_sq / (4*np.pi)

In [19]:
#qs = np.geomspace(constants.M_Z, 1e3 * constants.M_Z, 100)
#plt.plot(qs, 1 / (1 / (4*np.pi*constants.alpha_1) - 1 / (12*np.pi**2) * np.log(qs**2 / constants.M_Z**2)), 
#         label="direct U(1) running")
#g_2_sq = 4 * np.pi * alpha2_running(qs)
#e_EM_sq_M_Z = 4 * np.pi * (1 / 127)
#e_EM_sq = 1 / (1 / e_EM_sq_M_Z - 1 / (12*np.pi**2) * np.log(qs**2 / constants.M_Z**2))
#plt.plot(qs, g_2_sq * e_EM_sq / (g_2_sq - e_EM_sq),
#        label="indirect via SU(2) and U_EM(1) running")
#plt.axhline(constants.alpha_2 * constants.alpha_1 / (constants.alpha_1 + constants.alpha_2),)
#plt.axhline(1 / 127)
#plt.xscale("log")
#plt.yscale("log")

## SM RNG @ 3-Loops:
* https://arxiv.org/abs/1212.6829
* https://arxiv.org/abs/1303.4364
* https://arxiv.org/abs/1210.6873
* https://arxiv.org/abs/1310.3806

In [12]:
def calc_RNG_eq_Yukawa_rhs(ln_q, a_vec):
    # https://arxiv.org/abs/1212.6829
    
    q = np.exp(ln_q)
    # lam = lambda / (16 * np.pi**2) 
    
    at, ab, atau = a_vec
    a1 = 5 / 3 * alpha1_running(q)
    a2 = alpha2_running(q)
    a3 = alpha3_running(q)
    
    NG = 3 # number of generations
    
    h = 1
    lam = 
    
    #  beta-function for Top Yukawa *squared* : at = Yt**2/(16 Pi**2)
    bat = (
        at*((-17*a1)/20 - (9*a2)/4 + (3*ab)/2 - 8*a3 + (9*at)/2 + atau)*h**2 +
         at*h**4*((9*a1**2)/200 - (9*a1*a2)/20 - (35*a2**2)/4 + (7*a1*ab)/80 +
           (99*a2*ab)/16 - ab**2/4 + (19*a1*a3)/15 + 9*a2*a3 + 4*ab*a3 -
           (404*a3**2)/3 + (393*a1*at)/80 + (225*a2*at)/16 - (11*ab*at)/4 +
           36*a3*at - 12*at**2 + (15*a1*atau)/8 + (15*a2*atau)/8 + (5*ab*atau)/4 -
           (9*at*atau)/4 - (9*atau**2)/4 - 12*at*lam + 6*lam**2 + (29*a1**2*NG)/45 +
           a2**2*NG + (80*a3**2*NG)/9)
    )**0.5

    # beta-function for Bottom Yukawa *squared* : ab = Yb**2/(16 Pi**2)
    bab = (
        ab*(-a1/4 - (9*a2)/4 + (9*ab)/2 - 8*a3 + (3*at)/2 + atau)*h**2 +
         ab*h**4*((-29*a1**2)/200 - (27*a1*a2)/20 - (35*a2**2)/4 + (237*a1*ab)/80 +
           (225*a2*ab)/16 - 12*ab**2 + (31*a1*a3)/15 + 9*a2*a3 + 36*ab*a3 -
           (404*a3**2)/3 + (91*a1*at)/80 + (99*a2*at)/16 - (11*ab*at)/4 +
           4*a3*at - at**2/4 + (15*a1*atau)/8 + (15*a2*atau)/8 - (9*ab*atau)/4 +
           (5*at*atau)/4 - (9*atau**2)/4 - 12*ab*lam + 6*lam**2 - (a1**2*NG)/45 +
           a2**2*NG + (80*a3**2*NG)/9)
    )**0.5

    #  beta-function for Tau Yukawa *squared* : atau = Ytau**2/(16 Pi**2)
    batau = (
        atau*((-9*a1)/4 - (9*a2)/4 + 3*ab + 3*at + (5*atau)/2)*h**2 +
         atau*h**4*((51*a1**2)/200 + (27*a1*a2)/20 - (35*a2**2)/4 + (5*a1*ab)/8 +
           (45*a2*ab)/8 - (27*ab**2)/4 + 20*ab*a3 + (17*a1*at)/8 + (45*a2*at)/8 +
           (3*ab*at)/2 + 20*a3*at - (27*at**2)/4 + (537*a1*atau)/80 +
           (165*a2*atau)/16 - (27*ab*atau)/4 - (27*at*atau)/4 - 3*atau**2 -
        12*atau*lam + 6*lam**2 + (11*a1**2*NG)/5 + a2**2*NG)
    )**0.5
    
    # RNG eq: 
    # mu * d alpha / d mu =: beta(alpha)
    
    bs = np.array((bat, bab, batau)) / 2 # / 2 from definition of the beta functions
    return bs

In [13]:
masses = np.array((constants.m_top, constants.m_bottom, constants.m_tau))
Yukawa_couplings = masses / constants.higgs_vev
a_initial = Yukawa_couplings**2 / (16 * np.pi**2)

In [14]:
q_range = (constants.M_Z, 1e15)
solve_ivp(calc_RNG_eq_Yukawa_rhs, np.log(q_range), a_initial)

NameError: name 'lam' is not defined

In [ ]:
def yukawa_running(q, f, f_prime):
    return ????